In [ ]:
import os
import numpy as np
from tensorflow.keras.layers import Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD

# TRAINING_SET_DIR = os.getcwd() + '/drive/MyDrive/dog_pics/all_breeds_augmented/training_set_2/'
# VALIDATION_SET_DIR = os.getcwd() + '/drive/MyDrive/dog_pics/all_breeds_augmented/validation_set/'

TRAINING_SET_DIR = os.getcwd() + '/training_set/'
VALIDATION_SET_DIR = os.getcwd() + '/validation_set/'

IMAGE_WIDTH = 500
IMAGE_HEIGHT = 375

def run_model():
    # Use InceptionV3 model for feature extraction
    inception = InceptionV3(weights='imagenet', include_top=False)
    inception_output = inception.output
    inception_output = GlobalAveragePooling2D()(inception_output)

    # Add fully connected layer and output (prediction/classification) layer
    fc_layer = Dense(units=720, activation ='relu')(inception_output)
    predictions = Dense(120, activation ='softmax')(inception_output)
    model = Model(inputs = inception.input, outputs = predictions)

    # Freeze InceptionV3 layers
    for layer in inception.layers:
        layer.trainable = False

    # Summarise and compile model
    model.summary()
    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])

    # Generate training and validation data with augmentations
    gen_training = ImageDataGenerator()
    training_data = gen_training.flow_from_directory(
        directory=TRAINING_SET_DIR,
        target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
        class_mode='categorical',
    )

    gen_validation = ImageDataGenerator()
    validating_data = gen_validation.flow_from_directory(
        directory=VALIDATION_SET_DIR,
        target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
        class_mode='categorical',
    )

    # Fit model
    model.fit_generator(
        training_data,
        steps_per_epoch=training_data.n // training_data.batch_size,
        epochs=20,
        validation_data=validating_data,
        validation_steps=validating_data.n // validating_data.batch_size
    )


In [ ]:
!unzip "/content/drive/MyDrive/dog_pics/all_breeds_augmented/Archive.zip" -d "/content"